# Note, this *only* works for observing things in the future. Dealing with the pointing corrections in the analysis is not supported here.





# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
tstart = '2017-07-13T00:00:00'
tend = '2017-07-14T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [ ]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [2]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-07-13 00:22:04,2017-07-13 01:17:59
1,2017-07-13 01:58:44,2017-07-13 02:54:38
2,2017-07-13 03:35:24,2017-07-13 04:31:18
3,2017-07-13 05:12:03,2017-07-13 06:07:57
4,2017-07-13 06:48:43,2017-07-13 07:44:37


# Use SkyField to get the location of Jupiter for each orbit:

This puts the output into the provided text file. This shows the *aim* time that was used to determine the pointing. You should slew while the source is occulted.

In [9]:
from nustar_planning import moon
from imp import reload
reload(moon)
outfile = 'lunar_{0}_{1}_pointing.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
moon.position(orbits, load_path ='../../data',
                 parallax_correction=True,
                 outfile=outfile, show=True)
print("Output is stored in: {}".format(outfile))

Aim Time            RA         Dec
2017-07-13T00:22:04 337.44450  -10.85561
2017-07-13T00:22:04 337.46482  -10.91256
2017-07-13T00:22:04 337.01030  -10.90067
2017-07-13T00:22:04 336.35886  -10.80921
2017-07-13T00:22:04 335.90302  -10.67216
2017-07-13T01:58:44 338.27371  -10.61396
2017-07-13T01:58:44 338.30449  -10.66730
2017-07-13T01:58:44 337.85758  -10.65440
2017-07-13T01:58:44 337.20641  -10.56498
2017-07-13T01:58:44 336.74306  -10.43174
2017-07-13T03:35:24 339.10239  -10.36999
2017-07-13T03:35:24 339.14356  -10.41966
2017-07-13T03:35:24 338.70407  -10.40561
2017-07-13T03:35:24 338.05313  -10.31809
2017-07-13T03:35:24 337.58264  -10.18859
2017-07-13T05:12:03 339.93009  -10.12364
2017-07-13T05:12:03 339.98245  -10.16965
2017-07-13T05:12:03 339.55105  -10.15447
2017-07-13T05:12:03 338.90053  -10.06885
2017-07-13T05:12:03 338.42249  -9.94311
2017-07-13T06:48:43 340.75779  -9.87516
2017-07-13T06:48:43 340.82054  -9.91739
2017-07-13T06:48:43 340.39675  -9.90086
2017-07-13T06:48:43 339.74